In [ ]:
import datetime, os, time, pathlib
import numpy as np
import tensorflow as tf
from tensorflow.keras import losses, models, layers, optimizers, callbacks, utils, metrics
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7

In [ ]:
tf.config.list_physical_devices("GPU")

In [ ]:
data_dir = pathlib.Path("D:Dataset/dataset/랜드마크 이미지/training")

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg'))) # 총 이미지 개수
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

In [ ]:
val_size = int(image_count * 0.15)
train_ = list_ds.skip(val_size) # 20% 를 넘김
valid_ = list_ds.take(val_size) # 20% 를 가짐 
NUM_TRAIN = tf.data.experimental.cardinality(train_).numpy()
NUM_VALID = tf.data.experimental.cardinality(valid_).numpy()
print(tf.data.experimental.cardinality(train_).numpy())
print(tf.data.experimental.cardinality(valid_).numpy())

In [ ]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

In [ ]:
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_jpeg(img, channels=3)
    # resize the image to the desired size
    return tf.image.resize(img, [img_height, img_width])

In [ ]:
def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [ ]:
def build_eff_model():
    # Top
    x = layers.Input(shape=(img_height, img_width, 3))
    y = base_model(x)

    # Rebuild
    y = layers.GlobalAveragePooling2D(name="avg_pool")(y)
    y = layers.BatchNormalization()(y)
    top_dropout_rate = 0.2
    y = layers.Dropout(top_dropout_rate, name="top_dropout")(y)
    y = layers.Dense(84, activation="softmax", name="Pred")(y)
    
    # compile
    model = models.Model(x, y, name="EfficientNet")
    optimizer = optimizers.Adam(learning_rate=0.0001)
    loss = losses.SparseCategoricalCrossentropy()
    model.compile(optimizer=optimizer, loss=loss, metrics=["acc"])
    return model

In [ ]:
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [ ]:
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
class_names

In [ ]:
BATCH_SIZES = [32, 16, 16, 8, 4, 4, 2, 2]
SIZES = [224, 224, 224, 224, 224, 224, 224, 224]
AUTOTUNE = tf.data.experimental.AUTOTUNE
NAME = ["B0", "B1", "B2", "B3", "B4","B5","B6","B7"]
MODEL = [EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7]

for name,M, size, bs in zip(NAME, MODEL, SIZES, BATCH_SIZES):
    SIZE = size
    batch_size = bs
    img_height = SIZE
    img_width = SIZE
    image_size=(img_height, img_width)
    train_ds = train_.map(process_path, num_parallel_calls=AUTOTUNE)
    valid_ds = valid_.map(process_path, num_parallel_calls=AUTOTUNE)
    train_ds = configure_for_performance(train_ds)
    valid_ds = configure_for_performance(valid_ds)
    base_model = M(input_shape=(img_height,img_width,3), include_top=False, weights='imagenet')
    model = build_eff_model()
    ckp = callbacks.ModelCheckpoint(f"model/model_{name}.h5", save_best_only=True,  monitor="val_acc")
    early_stopping = callbacks.EarlyStopping(monitor="val_acc", patience=5)
    reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_acc", patience=3, factor=0.1)
    model.fit(train_ds,epochs=10,validation_data=valid_ds,callbacks=[ckp, early_stopping, reduce_lr])

In [ ]:
model.summary()